<a href="https://colab.research.google.com/github/willt08/rosa/blob/main/Rosa_V1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torch transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 64.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 38.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 32.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 109.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

In [2]:
from transformers import BertModel, BertTokenizer
import torch
import torch.nn as nn

# Define the ROSA model class
class Rosa(nn.Module):
    def __init__(self, model_name="bert-base-uncased", num_emotions=28):
        super().__init__()
        self.heart = BertModel.from_pretrained(model_name)
        self.grace = nn.Dropout(0.3)
        self.bloom = nn.Linear(self.heart.config.hidden_size, num_emotions)

    def forward(self, input_ids, attention_mask):
        petals = self.heart(input_ids=input_ids, attention_mask=attention_mask)
        pooled = petals.pooler_output
        softened = self.grace(pooled)
        logits = self.bloom(softened)
        return logits


In [3]:
# Download model weights (.pt) from your Hugging Face model repo
!wget https://huggingface.co/willt-dc/Rosa-V1/resolve/main/rosa.pt

--2025-07-06 04:14:13--  https://huggingface.co/willt-dc/Rosa-V1/resolve/main/rosa.pt
Resolving huggingface.co (huggingface.co)... 3.163.189.74, 3.163.189.90, 3.163.189.114, ...
Connecting to huggingface.co (huggingface.co)|3.163.189.74|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.hf.co/repos/5f/6a/5f6ab047273cacf29e23346367a908adbedfedd393c94ff329ca0e456f2a2109/794208af7d8065a964ee7327477f670fccd93b1c955263ebcc5b8ac0613f4ebb?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27rosa.pt%3B+filename%3D%22rosa.pt%22%3B&Expires=1751778853&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTc1MTc3ODg1M319LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmhmLmNvL3JlcG9zLzVmLzZhLzVmNmFiMDQ3MjczY2FjZjI5ZTIzMzQ2MzY3YTkwOGFkYmVkZmVkZDM5M2M5NGZmMzI5Y2EwZTQ1NmYyYTIxMDkvNzk0MjA4YWY3ZDgwNjVhOTY0ZWU3MzI3NDc3ZjY3MGZjY2Q5M2IxYzk1NTI2M2ViY2M1YjhhYzA2MTNmNGViYj9yZXNwb25zZS1jb250ZW50LWRpc3Bvc2l0aW9uPSoifV19&Sig

In [4]:
# Load tokenizer and model
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = Rosa(num_emotions=28)
model.load_state_dict(torch.load("rosa.pt", map_location=torch.device("cpu")))
model.eval()

# Run inference on your text
def predict(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    model_inputs = {
    "input_ids": inputs["input_ids"],
    "attention_mask": inputs["attention_mask"]
}
    with torch.no_grad():
        logits = model(**model_inputs)
        probs = torch.sigmoid(logits).squeeze()
    return probs

# Example
text = "And all I loved, I loved alone"
probs = predict(text)

# Emotion labels
emotion_labels = [
    "admiration", "amusement", "anger", "annoyance", "approval", "caring",
    "confusion", "curiosity", "desire", "disappointment", "disapproval",
    "disgust", "embarrassment", "excitement", "fear", "gratitude", "grief",
    "joy", "love", "nervousness", "optimism", "pride", "realization", "relief",
    "remorse", "sadness", "surprise", "neautral"
]

# Print nicely
for label, prob in zip(emotion_labels, probs.tolist()):
    print(f"{label:<15}: {prob:.4f}")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

admiration     : 0.0305
amusement      : 0.0076
anger          : 0.0059
annoyance      : 0.0065
approval       : 0.0187
caring         : 0.0072
confusion      : 0.0045
curiosity      : 0.0078
desire         : 0.0084
disappointment : 0.0062
disapproval    : 0.0073
disgust        : 0.0023
embarrassment  : 0.0022
excitement     : 0.0089
fear           : 0.0021
gratitude      : 0.0119
grief          : 0.0013
joy            : 0.0221
love           : 0.9037
nervousness    : 0.0014
optimism       : 0.0094
pride          : 0.0016
realization    : 0.0080
relief         : 0.0022
remorse        : 0.0051
sadness        : 0.0082
surprise       : 0.0054
neautral       : 0.0204


## Local Inference on GPU
Model page: https://huggingface.co/willt-dc/Rosa-V1

⚠️ If the generated code snippets do not work, please open an issue on either the [model repo](https://huggingface.co/willt-dc/Rosa-V1)
			and/or on [huggingface.js](https://github.com/huggingface/huggingface.js/blob/main/packages/tasks/src/model-libraries-snippets.ts) 🙏